In [49]:
import pandas as pd
import mysql.connector
from sqlalchemy import create_engine
import pymysql

In [5]:
dataset = pd.read_csv("geonames-all-cities-with-a-population-1000.csv", sep=";")
dataset.head(10)

In [74]:
exceed_count = (dataset['Population'] > 10000000).sum()
print("Number of times population exceeds 10,000,000:", exceed_count)

Number of times population exceeds 10,000,000: 17


In [78]:
try:
    # Connect to the MySQL database
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='abcd'
    )
    
    # Create a cursor
    cursor = connection.cursor()
    
    # Create the 'cities_populations' database if it doesn't exist
    create_database_query = "CREATE DATABASE IF NOT EXISTS cities_populations"
    cursor.execute(create_database_query)
    
    # Switch to the 'cities_populations' database
    use_database_query = "USE cities_populations"
    cursor.execute(use_database_query)
    
    # Create the 'cities_populations' table
    create_table_query = """
        CREATE TABLE IF NOT EXISTS cities_populations (
            city VARCHAR(255),
            country VARCHAR(255),
            population INT
        )
    """
    
    cursor.execute(create_table_query)
    
    # Insert the dataset into the 'cities_populations' table
    insert_query = "INSERT INTO cities_populations (city, country, population) VALUES (%s, %s, %s)"
    values = [tuple(row) for row in dataset[['Name', 'Country name EN', 'Population']].values]
    cursor.executemany(insert_query, values)
    
    # Query the database to find countries that don't host a megapolis
    query = """
    SELECT DISTINCT country
    FROM cities_populations
    WHERE country NOT IN (
        SELECT country
        FROM cities_populations
        WHERE population > 10000000
    )
    ORDER BY country
    """
    cursor.execute(query)
    result = cursor.fetchall()
    
    # Count the number of distinct countries
    count_query = """
        SELECT COUNT(DISTINCT country) AS count
        FROM cities_populations
    """
    cursor.execute(count_query)
    count_result = cursor.fetchone()
    country_count = count_result[0]
    
    # Save the result as a tabulated separated value file
    result_df = pd.DataFrame(result, columns=['country'])
    result_df.to_csv('result.tsv', sep='\t', index=False)
    
    # Close the cursor and connection
    cursor.close()
    connection.close()
    
    print("Query result saved to 'result.tsv'.")
    print("Number of distinct countries:", country_count)
    print(result_df)
    
except mysql.connector.Error as error:
    print("Error connecting to MySQL database:", error)

Query result saved to 'result.tsv'.
Number of distinct countries: 229
                      country
0                 Afghanistan
1                     Albania
2                     Algeria
3              American Samoa
4                     Andorra
..                        ...
214  West Bank and Gaza Strip
215            Western Sahara
216                     Yemen
217                    Zambia
218                  Zimbabwe

[219 rows x 1 columns]
